For loading the model and continue training

In [3]:
# from keras.models import load_model
# import segmentation_models_3D as sm
# from keras.optimizers import Adam
# import numpy as np

# # Step 1: Load the model
# model = load_model('./3D_model_vgg16_2epochs.h5', compile=False)

# # Step 2: Re-define loss and metrics
# n_classes = 8
# dice_loss = sm.losses.DiceLoss(class_weights=np.array([1/n_classes] * n_classes))  # or your custom weights
# focal_loss = sm.losses.CategoricalFocalLoss()
# total_loss = dice_loss + focal_loss

# metrics = [
#     sm.metrics.IOUScore(threshold=0.5),
#     sm.metrics.FScore(threshold=0.5)
# ]

# # Step 3: Recompile the model
# model.compile(
#     optimizer=Adam(learning_rate=1e-4),
#     loss=total_loss,
#     metrics=metrics
# )

# # Step 4: Continue training
# history = model.fit(
#     X_train,
#     y_train,
#     batch_size=8,
#     epochs=10,  # or more
#     validation_data=(X_test, y_test),
#     verbose=1
# )


In [4]:
#Load the pretrained model for testing and predictions.
from keras.models import load_model
my_model = load_model('./3D_model_vgg16_2epochs.h5', compile=False)

ON FULL VOLUME

In [5]:
import nibabel as nib
import numpy as np
from patchify import patchify

# Load the volume
nii_img = nib.load('./Data_Subset/images/ct_1003_image.nii.gz')
large_image = nii_img.get_fdata()  # shape: (H, W, D)

print("Original volume shape:", large_image.shape)

# Get original depth
depth = large_image.shape[2]
pad_depth = (64 - depth % 64) if depth % 64 != 0 else 0

# Pad along the depth axis (last axis)
padded_image = np.pad(large_image, ((0, 0), (0, 0), (0, pad_depth)), mode='constant', constant_values=0)
print("Padded volume shape:", padded_image.shape)

# Extract non-overlapping patches
patches = patchify(padded_image, (64, 64, 64), step=64)
print("Patches shape:", patches.shape)  # (x, y, z, 64, 64, 64)


Original volume shape: (512, 512, 206)
Padded volume shape: (512, 512, 256)
Patches shape: (8, 8, 4, 64, 64, 64)


In [7]:
import numpy
import tensorflow
import keras
import skimage
import patchify
import matplotlib
import sklearn
import nibabel
import segmentation_models_3D
import classification_models_3D
import efficientnet_3D

print("numpy:", numpy.__version__)
print("tensorflow:", tensorflow.__version__)
print("keras:", keras.__version__)
print("scikit-image:", skimage.__version__)
print("matplotlib:", matplotlib.__version__)
print("scikit-learn:", sklearn.__version__)
print("nibabel:", nibabel.__version__)
print("segmentation-models-3D:", segmentation_models_3D.__version__)
print("classification-models-3D:", classification_models_3D.__version__)
print("efficientnet-3D:", efficientnet_3D.__version__)


Segmentation Models: using `tf.keras` framework.
numpy: 1.24.3
tensorflow: 2.13.0
keras: 2.13.1
scikit-image: 0.21.0
matplotlib: 3.7.1
scikit-learn: 1.3.0
nibabel: 5.1.0
segmentation-models-3D: 1.0.4
classification-models-3D: 1.0.0
efficientnet-3D: 1.0.0


In [8]:
import segmentation_models_3D as sm

In [9]:

BACKBONE = 'vgg16'  
preprocess_input = sm.get_preprocessing(BACKBONE)


In [10]:
# Predict segmentation mask for each 3D patch
predicted_patches = []

for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        for k in range(patches.shape[2]):
            # Extract a single grayscale patch: shape (64, 64, 64)
            single_patch = patches[i, j, k, :, :, :]

            # Convert to 3-channel input: shape (64, 64, 64, 3)
            single_patch_3ch = np.stack((single_patch,) * 3, axis=-1)

            # Expand dimensions to make it batch-shaped: shape (1, 64, 64, 64, 3)
            single_patch_input = np.expand_dims(single_patch_3ch, axis=0)

            # Preprocess input according to VGG16
            single_patch_input = preprocess_input(single_patch_input)

            # Predict segmentation output: shape (1, 64, 64, 64, 8)
            prediction = my_model.predict(single_patch_input)

            # Convert probabilities to class labels: shape (64, 64, 64)
            prediction_argmax = np.argmax(prediction, axis=4)[0]

            # Append prediction
            predicted_patches.append(prediction_argmax)


1/1 [==============================] - 1s 1s/step


In [11]:
# Convert and reshape prediction list
predicted_patches = np.array(predicted_patches)
print("Predicted patches shape (flat):", predicted_patches.shape)

Predicted patches shape (flat): (256, 64, 64, 64)


In [12]:
predicted_patches_reshaped = np.reshape(predicted_patches,
    (patches.shape[0], patches.shape[1], patches.shape[2], 64, 64, 64)
)
print("Reshaped to patch grid:", predicted_patches_reshaped.shape)

Reshaped to patch grid: (8, 8, 4, 64, 64, 64)


In [13]:
from patchify import patchify, unpatchify

In [14]:
# Reconstruct segmented volume
reconstructed_image = unpatchify(predicted_patches_reshaped, padded_image.shape)
reconstructed_image = reconstructed_image[:, :, :depth]  # Remove padding
print("Final reconstructed shape:", reconstructed_image.shape)

Final reconstructed shape: (512, 512, 206)


In [15]:
from skimage.io import imsave

# Save basic segmented volume
reconstructed_image = reconstructed_image.astype(np.uint8)
imsave('./segmented.tif', reconstructed_image)
print("Saved raw segmented image")

Saved raw segmented image


C:\Users\manvi sharma\AppData\Local\Temp\ipykernel_18984\4220865166.py:5: UserWarning: ./segmented.tif is a low contrast image
  imsave('./segmented.tif', reconstructed_image)


In [16]:
# Create binary masks for each of the 8 classes
num_segments = 8
final = np.zeros((*reconstructed_image.shape, num_segments), dtype=np.int8)

for i in range(num_segments):
    final[..., i] = (reconstructed_image == i).astype(np.int8)

In [17]:

#Use APEER OMETIFF library to read and write multidimensional images
%pip install apeer-ometiff-library

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\manvi sharma\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [18]:
from apeer_ometiff_library import io

In [19]:
# Convert to OMETIFF format (T, Z, C, X, Y)
final = np.expand_dims(final, axis=0)  # Add time axis T=1
final = np.swapaxes(final, 2, 4)       # Move channels to 3rd dimension
print("Shape for OMETIFF (T, Z, C, X, Y):", final.shape)

# Save as OMETIFF
io.write_ometiff("./segmented_multi_channel.ome.tiff", final)
print("Multi-channel segmented OMETIFF saved successfully!")

Shape for OMETIFF (T, Z, C, X, Y): (1, 512, 8, 206, 512)
Multi-channel segmented OMETIFF saved successfully!


In [20]:
print("Shape of the segmented volume is: T, Z, C, X, Y ", final.shape)
print(final.dtype)

Shape of the segmented volume is: T, Z, C, X, Y  (1, 512, 8, 206, 512)
int8
